
# AWS Lab Exercise — Private Subnet (Console Based)

### Objective
In this exercise, you will extend your earlier setup (Public Subnet + Internet Gateway) by creating a **Private Subnet** and launching an EC2 instance (**EC2-B**) inside it.  
This instance will not have outbound internet connectivity, demonstrating how private subnets are isolated from the public internet.

---

### Architecture Diagram
```
Region: ap-south-1 (Mumbai)
└── VPC (10.100.0.0/16)
    ├── Public Subnet (10.100.0.0/24) → EC2-A (Public + Private IP)
    ├── Private Subnet (10.100.11.0/24) → EC2-B (Private IP only)
    └── Internet Gateway (attached to VPC)
```
Public subnet route table:
- 10.100.0.0/16 → Local  
- 0.0.0.0/0 → igw-xxxxxx  

Private subnet route table:
- 10.100.0.0/16 → Local  

---

### High-Level Steps
1. Create a new private subnet  
2. Create and associate a route table for the private subnet  
3. Launch EC2-B in the private subnet  
4. Connect from EC2-A (public) to EC2-B (private) via SSH  
5. Try to reach the internet from EC2-B (expected to fail)

---



## Step 1: Create a Private Subnet

1. In the **AWS Management Console**, navigate to **VPC Service** → **Subnets** → click **Create Subnet**.  
2. **Select VPC ID:** `VPC-A`.  
3. For **Subnet name**, enter `VPC-A-Private`.  
4. Choose the same Availability Zone (`ap-south-1a`).  
5. Enter **IPv4 CIDR block:** `10.100.11.0/24`.  
6. Click **Create subnet**.  

> The subnet does **not** need a public IP auto-assignment because it is intended to remain private.



## Step 2: Create a Route Table for the Private Subnet

1. Navigate to **Route Tables** → click **Create route table**.  
2. **Name tag:** `VPC-A-Private-RT`.  
3. Select **VPC:** `VPC-A`.  
4. Click **Create route table**.  
5. Select the newly created route table → **Subnet associations** → click **Edit subnet associations**.  
6. Check **VPC-A-Private** subnet and click **Save associations**.  

> This private route table only includes a **local** route by default.  
> It does not have a route to the Internet Gateway, ensuring isolation.



## Step 3: Launch EC2 Instance in the Private Subnet

1. Navigate to **EC2 Service** → **Instances** → click **Launch Instance**.  
2. **Name:** `EC2-B`.  
3. Select **Amazon Linux** as the AMI (default).  
4. **Instance type:** `t2.micro`.  
5. **Key pair:** Select the same key you created earlier (for EC2-A).  
6. Under **Network settings → Edit**:
   - **VPC:** `VPC-A`  
   - **Subnet:** `VPC-A-Private`  
   - Ensure **Auto-assign Public IP** is **disabled**.  
7. Under **Firewall (security group)** → click **Create security group**:  
   - **Name:** `EC2-B-SG`  
   - Add inbound rule: **SSH (port 22)** → Source: `10.100.0.0/16`  
   - Add inbound rule: **ICMP (ping)** → Source: `10.100.0.0/16`  
8. Keep default **storage** (8 GiB, gp3).  
9. Click **Launch Instance**.  

> Wait until EC2-B reaches the `running` state.



## Step 4: Connect from EC2-A to EC2-B (Private IP)

1. Connect to your **EC2-A (Public)** instance via SSH using the `.pem` key file.  
   ```bash
   ssh -i your-key.pem ec2-user@<EC2-A-Public-IP>
   ```

2. Once connected to EC2-A, ping EC2-B’s private IP to verify internal connectivity.  
   ```bash
   ping 10.100.11.x
   ```

> If the ping succeeds, it confirms internal communication between subnets within the VPC.



## Step 5: Transfer SSH Key to EC2-A

1. On EC2-A, create a new key file named `key.pem` using any text editor.  
   ```bash
   nano key.pem
   ```

2. Paste your existing private key contents into the file and save it (`Ctrl+O`, `Enter`, `Ctrl+X`).  
3. Change file permissions:  
   ```bash
   chmod 400 key.pem
   ```



## Step 6: SSH into EC2-B (from EC2-A)

1. From EC2-A terminal, connect to EC2-B using its **private IP**:  
   ```bash
   ssh -i key.pem ec2-user@10.100.11.x
   ```

2. Once logged in to EC2-B, you are now inside a private subnet instance.

> Note: EC2-B cannot access the internet as the private route table lacks a route to the Internet Gateway.



## Step 7: Test Internet Access (Expected to Fail)

Inside **EC2-B**, try the following commands:

```bash
ping google.com
wget https://google.com
```

You should observe no response, confirming that the **Private Subnet** does not have outbound internet connectivity.

> [Expected Behavior] — Above commands should fail because there is no outbound route (0.0.0.0/0 → IGW).



---

## Reflection

✅ What makes this subnet *private*?  
→ It has no Internet Gateway route and does not assign public IPs to instances.  

✅ How could we provide controlled internet access?  
→ By adding a **NAT Gateway** in the public subnet and updating the private subnet's route table to use it.  

✅ Key Learning  
You learned how to isolate workloads inside a private subnet and access them indirectly through a bastion or public EC2 instance.

---
